# 📊 GPT-assisted Report Type Classification
This notebook reads PDF reports, extracts front/back pages, and uses OpenAI GPT API to classify the report type and sustainability section presence.

In [1]:
from openai import OpenAI
from pathlib import Path
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm
import json
import os
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Paths
PDF_DIR = Path("pdf_folder")  # Ensure this path contains your PDFs
OUTPUT_PATH = Path("output/report_type_gpt_results.csv")

In [2]:
# 重新加载必要模块和OCR补救函数（环境已重置）
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import os

# OCR 补救函数：用于从首页和末页图像提取文本
def extract_text_with_ocr(pdf_path, front_n=5, back_n=5, dpi=300):
    try:
        with tempfile.TemporaryDirectory() as path:
            # 将 PDF 页转换为图像
            images = convert_from_path(pdf_path, dpi=dpi, output_folder=path)
            total_pages = len(images)
            selected_pages = []

            for i in range(min(front_n, total_pages)):
                selected_pages.append(images[i])

            for i in range(max(0, total_pages - back_n), total_pages):
                selected_pages.append(images[i])

            # 用 OCR 提取文本
            text_parts = [pytesseract.image_to_string(img) for img in selected_pages]
            return "\n".join(text_parts)
    except Exception as e:
        return f"OCR ERROR: {e}"


In [3]:
# Extract front/back page text from a PDF
# def extract_front_back_text(pdf_path, front_n=5, back_n=5):
#     try:
#         doc = fitz.open(pdf_path)
#         texts = []
#         for i in range(min(front_n, len(doc))):
#             texts.append(doc[i].get_text())
#         for i in range(max(0, len(doc) - back_n), len(doc)):
#             texts.append(doc[i].get_text())
#         doc.close()
#         return "\n".join(texts)
#     except Exception as e:
#         return f"ERROR: {e}"
def extract_front_back_text(pdf_path, front_n=5, back_n=5):
    try:
        doc = fitz.open(pdf_path)
        texts = []
        for i in range(min(front_n, len(doc))):
            texts.append(doc[i].get_text())
        for i in range(max(0, len(doc) - back_n), len(doc)):
            texts.append(doc[i].get_text())
        doc.close()
        full_text = "\n".join(texts)
        if len(full_text.strip()) < 50:
            raise ValueError("Empty or invalid text, fallback to OCR.")
        return full_text
    except Exception as e:
        print(f"⚠️ PyMuPDF failed on {pdf_path.name}, switching to OCR...")
        return extract_text_with_ocr(pdf_path)

In [4]:
# Use GPT to classify report type and sustainability section
def classify_report_type(text):
    system_prompt = "You are a helpful assistant."
    user_prompt = f"""
Given the following content extracted from a corporate report (first and last pages), classify the report into one of the following types:
- "annual report" (financial content only)
- "sustainability report" (focused on ESG or sustainability)
- "integrated report" (combination of financial and sustainability)
- "other"

Also determine whether the report includes a substantial sustainability section (e.g. with a dedicated chapter or multiple references to sustainability, SDGs, GRI, etc.).

Return a JSON object with the following structure:

{{
  "report_type": "...",
  "has_sustainability_section": true/false
}}

Content:
{text[:8000]}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        reply = response.choices[0].message.content
        parsed = json.loads(reply)
        return parsed.get("report_type", ""), parsed.get("has_sustainability_section", "")
    except Exception as e:
        return f"ERROR: {e}", ""

In [5]:
# Loop through PDFs and classify
pdf_results = []
for pdf_file in tqdm(os.listdir(PDF_DIR)):
    if not pdf_file.endswith(".pdf"):
        continue
    pdf_path = PDF_DIR / pdf_file
    text = extract_front_back_text(pdf_path)
    if text.startswith("ERROR"):
        pdf_results.append({"filename": pdf_file, "report_type": "ERROR", "has_sustainability_section": ""})
        continue
    report_type, sustainability = classify_report_type(text)
    pdf_results.append({
        "filename": pdf_file,
        "report_type": report_type,
        "has_sustainability_section": sustainability
    })

 20%|█▉        | 251/1277 [03:12<10:49,  1.58it/s]

⚠️ PyMuPDF failed on Unknown_adbi-managing-transition-low-carbon-economy_087is5zy.pdf, switching to OCR...


 20%|██        | 256/1277 [03:16<13:59,  1.22it/s]

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed



 20%|██        | 257/1277 [03:17<12:49,  1.33it/s]

...

 97%|█████████▋| 1239/1277 [15:48<00:29,  1.28it/s]

⚠️ PyMuPDF failed on Titan_Company_Ltd_Annual20Report202013_p4r8w07u.pdf, switching to OCR...


 99%|█████████▉| 1269/1277 [16:11<00:05,  1.50it/s]

⚠️ PyMuPDF failed on BASF_SE_2012_BASF_Report_lmq79gwn.pdf, switching to OCR...


100%|██████████| 1277/1277 [16:17<00:00,  1.31it/s]


In [7]:
# Save results
result_df = pd.DataFrame(pdf_results)
OUTPUT_PATH = Path("output/report_type_gpt_results1.csv")
result_df.to_csv(OUTPUT_PATH, index=False)
result_df.head()

,filename,report_type,has_sustainability_section
0,Unknown_8f57f855-11bb-496d-9916-91ff88cb537b_s...,annual report,False
1,Toyota_Industries_Corp_environment2004_40h96hj...,sustainability report,True
2,Knoll_Inc_Knoll_Enviro_2008_gqetdkb7.pdf,sustainability report,True
3,Intel_Corp__fwws0wtm.pdf,integrated report,True
4,Unknown_2020_SEBANG20SUSTAINABILITY20REPORT_EN...,integrated report,True


In [8]:
from pdf2image import convert_from_path
import pytesseract
import os
from tqdm import tqdm
import warnings

# 用户目录设置
pdf_dir = "/Users/zhangjingyu/Desktop/课件/IRP/starting/pdf_folder"
output_dir = os.path.join(pdf_dir, "ocr_outputs")
os.makedirs(output_dir, exist_ok=True)

# Tesseract 配置（可选：限制语言为英文）
ocr_config = "--psm 1 -l eng"

# 处理失败的 PDF 列表（你提供的）
failed_files = [
    "Lincoln_Electric_Holdings_Inc_le-air-treatment-catalogue-eng_8cadb8rq.pdf",
    "SATS_Ltd_sats-sustainability-report_88g98j6g.pdf",
    "Indusind_BankIndia_Business-Responsibility-and-Sustainability-Report-BRSR-FY2021-22_xw1dg5fb.pdf",
    "Simona_AG_SIMONA_2021_GB21_englisch_54emvime.pdf",
    "Unknown_2020-2021-corporate-responsibility-report_91a20q9k.pdf",
    "Grasim_Industries_Ltd_Business20Responsibility20and20Sustainability20Report_3yg2omk9.pdf",
    "Nichols_PLC_2020-Nichols-plc-AR_wulbuhbx.pdf",
    "Unknown_2021_Target_Corporate-Responsibility-Report_z0za8brg.pdf",
    "Bollore_SE_publication_344_en_5159im40.pdf",
    "Sitowise_Group_Oyj_sitowise-annual-report-sustainability-report-2021pdf_x563vwz2.pdf",
    "Unknown_2023042704942_wzv2kg92.pdf"
]

# OCR 主程序
for fname in tqdm(failed_files, desc="OCR Processing"):
    pdf_path = os.path.join(pdf_dir, fname)
    output_path = os.path.join(output_dir, fname.replace(".pdf", ".txt"))

    try:
        images = convert_from_path(pdf_path, dpi=300)
        text_all = []

        for img in images[:10]:  # 控制页数，可改为 [:5]+[-5:] 仅读前后页
            text = pytesseract.image_to_string(img, config=ocr_config)
            text_all.append(text)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write("\n".join(text_all))

    except Exception as e:
        warnings.warn(f"⚠️ OCR failed for {fname}: {e}")

OCR Processing:   0%|          | 0/11 [00:00<?, ?it/s]/var/folders/9z/mm8bj8yn2yj6rk7vdlhpt07h0000gn/T/ipykernel_26451/2052024968.py:47: UserWarning: ⚠️ OCR failed for Lincoln_Electric_Holdings_Inc_le-air-treatment-catalogue-eng_8cadb8rq.pdf: Unable to get page count. Is poppler installed and in PATH?
  warnings.warn(f"⚠️ OCR failed for {fname}: {e}")
/var/folders/9z/mm8bj8yn2yj6rk7vdlhpt07h0000gn/T/ipykernel_26451/2052024968.py:47: UserWarning: ⚠️ OCR failed for SATS_Ltd_sats-sustainability-report_88g98j6g.pdf: Unable to get page count. Is poppler installed and in PATH?
  warnings.warn(f"⚠️ OCR failed for {fname}: {e}")
/var/folders/9z/mm8bj8yn2yj6rk7vdlhpt07h0000gn/T/ipykernel_26451/2052024968.py:47: UserWarning: ⚠️ OCR failed for Indusind_BankIndia_Business-Responsibility-and-Sustainability-Report-BRSR-FY2021-22_xw1dg5fb.pdf: Unable to get page count. Is poppler installed and in PATH?
  warnings.warn(f"⚠️ OCR failed for {fname}: {e}")
/var/folders/9z/mm8bj8yn2yj6rk7vdlhpt07h0000gn/T

In [10]:
from pdf2image import convert_from_path
import pytesseract

path = "/Users/zhangjingyu/Desktop/课件/IRP/starting/pdf_folder/Bollore_SE_publication_344_en_5159im40.pdf"
images = convert_from_path(path, dpi=300)
texts = [pytesseract.image_to_string(img) for img in images[:3]]  # 仅测前3页
print("\n--- PAGE TEXT ---\n", texts[0][:1000])  # 打印前1000字符


--- PAGE TEXT ---
 Bolloré

Tour Bolloré

31-32, quai de Dion-Bouton
92811 Puteaux Cedex — France
Tel.: + 33 (0)1 46 96 44 33

Fax: + 33 (0)1 46 96 44 22

www.bollore.com

Bollore\

00
fe}
oO
ia
KE
or
O
at
Ww
or
a
<
=)
Z
Z
<

BOLLORE

Annual report
Bollore

2008

Bollore\

